# Spotify popularity prediction using Machine Learning based on various attributes of the song

## First of all import all the necessary modules
* Pandas for creating the dataframe
* MinMaxScaler for scaling the attributes
* LinearRegression for predicitng the exact degree of popularity
* LogisticRegression for Classifying into NotPopular, Neutral, Popular
* Decision Tree from classifiaction into Popular('Yes','No')
* Accuracy_score to check the accuracy of the trained model

In [2]:
import numpy as np
import pandas as pd
import tensorflow as tf
from keras.models import Sequential
from sklearn.linear_model import LinearRegression,LogisticRegression
from sklearn.metrics import mean_absolute_error,mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import MinMaxScaler
import joblib

## Read the .csv file into a Pandas Dataframe and remove TrackID and Index

In [3]:
df = pd.read_csv('spotify.csv')
print(df.columns)
print(df.head())
df = df.drop(['Unnamed: 0','track_id'], axis='columns')
print(df.columns)
exploded_df = df.explode('track_genre')

Index(['Unnamed: 0', 'track_id', 'artists', 'album_name', 'track_name',
       'popularity', 'duration_ms', 'explicit', 'danceability', 'energy',
       'key', 'loudness', 'mode', 'speechiness', 'acousticness',
       'instrumentalness', 'liveness', 'valence', 'tempo', 'time_signature',
       'track_genre'],
      dtype='object')
   Unnamed: 0                track_id                 artists   
0           0  5SuOikwiRyPMVoIQDJUgSV             Gen Hoshino  \
1           1  4qPNDBW1i3p13qLCt0Ki3A            Ben Woodward   
2           2  1iJBSr7s7jYXzM8EGcbK5b  Ingrid Michaelson;ZAYN   
3           3  6lfxq3CG4xtTiEg7opyCyx            Kina Grannis   
4           4  5vjLSffimiIP26QG5WcN2K        Chord Overstreet   

                                          album_name   
0                                             Comedy  \
1                                   Ghost (Acoustic)   
2                                     To Begin Again   
3  Crazy Rich Asians (Original Motion Picture Sou...

## Exploded df with all the genres having a separate column and boolean values depending on whether they belong to that genre or not

In [4]:
genres = exploded_df['track_genre'].unique()
for genre in genres:
    df[genre] = df['track_genre'].apply(lambda x: 1 if genre in x else 0)
df.drop(columns=['track_genre'], inplace=True)
print(df.columns)

/var/folders/tr/7vyd9xfn4817p8wdpzlw862m0000gn/T/ipykernel_97542/932512162.py:3: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[genre] = df['track_genre'].apply(lambda x: 1 if genre in x else 0)
/var/folders/tr/7vyd9xfn4817p8wdpzlw862m0000gn/T/ipykernel_97542/932512162.py:3: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[genre] = df['track_genre'].apply(lambda x: 1 if genre in x else 0)
/var/folders/tr/7vyd9xfn4817p8wdpzlw862m0000gn/T/ipykernel_97542/932512162.py:3: PerformanceWarning: DataFrame is highly fragmented.  This i

Index(['artists', 'album_name', 'track_name', 'popularity', 'duration_ms',
       'explicit', 'danceability', 'energy', 'key', 'loudness',
       ...
       'spanish', 'study', 'swedish', 'synth-pop', 'tango', 'techno', 'trance',
       'trip-hop', 'turkish', 'world-music'],
      dtype='object', length=132)


/var/folders/tr/7vyd9xfn4817p8wdpzlw862m0000gn/T/ipykernel_97542/932512162.py:3: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[genre] = df['track_genre'].apply(lambda x: 1 if genre in x else 0)
/var/folders/tr/7vyd9xfn4817p8wdpzlw862m0000gn/T/ipykernel_97542/932512162.py:3: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[genre] = df['track_genre'].apply(lambda x: 1 if genre in x else 0)
/var/folders/tr/7vyd9xfn4817p8wdpzlw862m0000gn/T/ipykernel_97542/932512162.py:3: PerformanceWarning: DataFrame is highly fragmented.  This i

In [5]:
print(df.size)

15048000


In [6]:
print(df.shape)

(114000, 132)


## Insert a separate column for Result which indicates whether the song was popular or not

In [7]:
result = []
for i in df['popularity']:
    if i>=0.6:
        result.append('Yes')
    else:
        result.append('No')
print(result[:5])
df['Result'] = result
df.shape

['Yes', 'Yes', 'Yes', 'Yes', 'Yes']


/var/folders/tr/7vyd9xfn4817p8wdpzlw862m0000gn/T/ipykernel_97542/445262401.py:8: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df['Result'] = result


(114000, 133)

## Insert another column of popularity index(pop_index) separting the tracks into Popular, Neutral or NotPopular (-1,0,1)

In [8]:
pop_index = []
for i in df['popularity']:
    if i<0.33:
        pop_index.append(-1)
    elif i>=0.33 and 0.67:
        pop_index.append(0)
    else:
        pop_index.append(1)
df['pop_index'] = pop_index

/var/folders/tr/7vyd9xfn4817p8wdpzlw862m0000gn/T/ipykernel_97542/1647889354.py:9: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df['pop_index'] = pop_index


In [9]:
print(df.describe())

          popularity   duration_ms   danceability         energy   
count  114000.000000  1.140000e+05  114000.000000  114000.000000  \
mean       33.238535  2.280292e+05       0.566800       0.641383   
std        22.305078  1.072977e+05       0.173542       0.251529   
min         0.000000  0.000000e+00       0.000000       0.000000   
25%        17.000000  1.740660e+05       0.456000       0.472000   
50%        35.000000  2.129060e+05       0.580000       0.685000   
75%        50.000000  2.615060e+05       0.695000       0.854000   
max       100.000000  5.237295e+06       0.985000       1.000000   

                 key       loudness           mode    speechiness   
count  114000.000000  114000.000000  114000.000000  114000.000000  \
mean        5.309140      -8.258960       0.637553       0.084652   
std         3.559987       5.029337       0.480709       0.105732   
min         0.000000     -49.531000       0.000000       0.000000   
25%         2.000000     -10.013000       

## Drop the rows with null values to avoid errors

In [10]:
nan_count = df.isnull().sum().sum()
print(nan_count)
df = df.dropna()
nan_count = df.isnull().sum().sum()
print(nan_count)
# df.dtypes
df['artists'] = pd.factorize(df['artists'])[0]
df['album_name'] = pd.factorize(df['album_name'])[0]
df['track_name'] = pd.factorize(df['track_name'])[0]
df.dtypes

3
0


artists         int64
album_name      int64
track_name      int64
popularity      int64
duration_ms     int64
                ...  
trip-hop        int64
turkish         int64
world-music     int64
Result         object
pop_index       int64
Length: 134, dtype: object

## Scale the data of specified columns using MinMaxScaler in a range of (0,1)

In [11]:
columns_to_be_scaled = ['artists','album_name','track_name','popularity','duration_ms','tempo','time_signature']
scaler = MinMaxScaler(feature_range=(0,1))
df[columns_to_be_scaled] = scaler.fit_transform(df[columns_to_be_scaled])
# print(df.head)
df.describe()

,artists,album_name,track_name,popularity,duration_ms,danceability,energy,key,loudness,mode,...,study,swedish,synth-pop,tango,techno,trance,trip-hop,turkish,world-music,pop_index
count,113999.000000,113999.000000,113999.000000,113999.000000,113999.000000,113999.000000,113999.000000,113999.000000,113999.000000,113999.000000,...,113999.000000,113999.000000,113999.000000,113999.000000,113999.000000,113999.000000,113999.000000,113999.000000,113999.000000,113999.000000
mean,0.443883,0.409699,0.443528,0.332388,0.041969,0.566801,0.641383,5.309126,-8.258950,0.637558,...,0.008772,0.008772,0.008772,0.008772,0.026316,0.008772,0.008772,0.008772,0.008772,-0.140519
std,0.291970,0.294490,0.289543,0.223050,0.020521,0.173543,0.251530,3.559999,5.029357,0.480708,...,0.093248,0.093248,0.093248,0.093248,0.160074,0.093248,0.093248,0.093248,0.093248,0.347526
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-49.531000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-1.000000
25%,0.181448,0.139435,0.192679,0.170000,0.031648,0.456000,0.472000,2.000000,-10.013000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,0.442741,0.361402,0.413602,0.350000,0.039077,0.580000,0.685000,5.000000,-7.004000,1.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,0.677981,0.648225,0.684344,0.500000,0.048371,0.695000,0.854000,8.000000,-5.003000,1.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,1.000000,1.000000,1.000000,1.000000,1.000000,0.985000,1.000000,11.000000,4.532000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.000000


## Split the dataframe for training and testing using train_test_split()
## I have 3 separate train and test tables for 3 types of predicitons, y for Linear Regression, z for DecisonTreeClassifier and p for LogisticRegression

In [12]:
x = df.drop(['popularity','Result'], axis='columns')
X = df.drop(['pop_index'],axis='columns')
y = df['popularity']
z = df['Result']
p = df['pop_index']
X_train,X_test,x_train, x_test, y_train, y_test, z_train,z_test,p_train, p_test= train_test_split(X,x,y,z,p,test_size=0.3, random_state=101)
print(x_test.shape)
print(x_train.shape)
print(y_train.shape)
print(y_test.shape)
print(z_train.shape)
print(z_test.shape)

(34200, 132)
(79799, 132)
(79799,)
(34200,)
(79799,)
(34200,)


In [13]:
print(y_test[0])

0.73


In [14]:
print(x_test.describe())

            artists    album_name    track_name   duration_ms  danceability   
count  34200.000000  34200.000000  34200.000000  34200.000000  34200.000000  \
mean       0.444223      0.409479      0.444009      0.041897      0.568453   
std        0.291066      0.293743      0.288424      0.020726      0.172880   
min        0.000000      0.000000      0.000000      0.000000      0.000000   
25%        0.181893      0.141034      0.195600      0.031597      0.459000   
50%        0.446669      0.363323      0.415253      0.039056      0.582000   
75%        0.676899      0.645493      0.682476      0.048247      0.696000   
max        1.000000      1.000000      0.999973      0.903037      0.981000   

             energy           key      loudness          mode   speechiness   
count  34200.000000  34200.000000  34200.000000  34200.000000  34200.000000  \
mean       0.644422      5.290117     -8.212494      0.636930      0.085167   
std        0.249364      3.558205      4.971696    

In [15]:
len(df['artists'].unique())

31437

## LinearRegression and it's accuracy

In [16]:
lr = LinearRegression()
lr.fit(x_train,y_train)
lr.score(x_test,y_test)

0.5910678017920459

In [17]:
y_pred=lr.predict(x_test)
lr.score(x_test,y_test)

0.5910678017920459

## Checking the mean squared error

In [18]:
r2 = mean_squared_error(y_pred,y_test)
print(r2)

0.02037187762193875


## DecisionTreeClassifer for classifying tracks into Popular or Not Popular

In [19]:
clf_entropy = DecisionTreeClassifier(criterion="entropy",random_state=100
                                    ,max_depth=3, min_samples_leaf=5)
clf_entropy.fit(x_train,z_train)

DecisionTreeClassifier(criterion='entropy', max_depth=3, min_samples_leaf=5,
                       random_state=100)

In [20]:
z_pred = clf_entropy.predict(x_test)
print(z_pred)

['Yes' 'No' 'Yes' ... 'Yes' 'Yes' 'Yes']


## Check the score of the model

In [21]:
print(accuracy_score(z_test,z_pred))

1.0


## Save the model using joblib to use later

In [22]:
joblib.dump(clf_entropy, 'spotify_popularity_pred.joblib')

['spotify_popularity_pred.joblib']

## LabelEncoder for encoding the 'Yes' and 'No' values in the train and test dataset 
## LogisticalRegression model is used for classifying into 'Popular','Neutral','Not Popular' and accuracy is checked
## LogisticalRegression model is saved using joblib for future use

In [23]:
from sklearn.preprocessing import LabelEncoder
encoder = LabelEncoder()
X_train['Result'] = encoder.fit_transform(X_train['Result'])
X_test['Result'] = encoder.fit_transform(X_test['Result'])
model = LogisticRegression(max_iter = 200)
model.fit(X_train,p_train)
p_pred = model.predict(X_test)
print(accuracy_score(p_pred,p_test))
joblib.dump(model,'logistical_spotify.joblib')

1.0


['logistical_spotify.joblib']